In [81]:
import pandas as pd
import numpy as np
import pathlib as pl
import sklearn

In [82]:
path = './links_fiqa.csv'

df = pd.read_csv(path, encoding = "ISO-8859-1")
# df

In [83]:
sources = []

from urllib.parse import urlparse

for i in range(len(df.index)):
    domain = urlparse(df['links'][i]).netloc
    sources.append(domain)

df['source'] = pd.Series(sources)
# df

In [84]:
# df.source.value_counts().sort_values().plot(kind = 'barh')
pd.options.display.max_rows = 4000
df.source.value_counts().sort_values()

www.beveragedaily.com                   1
uk.sports.yahoo.com                     1
www.chicagobusiness.com                 1
simplywall.st                           1
www.abrdn.com                           1
apnews.com                              1
static.aviva.io                         1
www.prnewswire.com                      1
www.tescoplc.com                        1
globalinsolvency.com                    1
www.ship-technology.com                 1
www.offshore-energy.biz                 1
github.com                              1
www.pharmatimes.com                     1
www.intertek.com                        1
group.legalandgeneral.com               1
europa.eu                               1
www.fool.com                            1
www.bighospitality.co.uk                1
www.insightdiy.co.uk                    1
www.bp.com                              1
coca-colafemsa.com                      1
www.hammerson.com                       1
www.aol.co.uk                     

In [85]:
df = df[df['source'].str.contains("reuters")]
df = df.reset_index(drop=True)

In [86]:
# df

In [87]:
start = 40
end = 50

df = df[start:end]
df = df.reset_index(drop=True)
df

,sentence,target,sentiment_score,links,source
0,Doubts grow over GlaxoSmithKline's $6 bln capi...,GlaxoSmithKline,-0.221,https://www.reuters.com/article/gsk-capital-id...,www.reuters.com
1,easyJet leads Britain's FTSE lower as global b...,easyJet,-0.495,https://www.reuters.com/article/markets-stocks...,www.reuters.com
2,Imperial Tobacco Adds Quarterly Dividends Afte...,Imperial Tobacco,0.336,https://www.reuters.com/article/imperial-tobac...,www.reuters.com
3,SABMiller buys Meantime to quench thirst for c...,SABMiller,0.296,https://www.reuters.com/article/us-meantime-m-...,www.reuters.com
4,Retailers Kingfisher and Sports Direct rise in...,Sports Direct,0.459,https://jp.reuters.com/article/markets-stocks-...,jp.reuters.com
5,"GlaxoSmithKline targets growth across units, s...",GlaxoSmithKline,0.229,https://www.reuters.com/article/gsk-results-id...,www.reuters.com
6,"RPT-Old Mutual Q1 gross sales beat forecasts, ...",Old Mutual,0.724,https://www.reuters.com/article/old-mutual-res...,www.reuters.com
7,UPDATE 3-BP settles oil spill-related claims w...,BP,-0.121,https://www.reuters.com/article/halliburton-bp...,www.reuters.com
8,UPDATE 1-Cypress Semiconductor offers to buy I...,Integrated Silicon Solution,0.461,https://www.reuters.com/article/integrated-sil...,www.reuters.com
9,"UPDATE 1-Nomura, RBS must pay $806 mln in mort...",RBS,-0.690,https://www.reuters.com/article/fhfa-nomura-hl...,www.reuters.com


In [88]:
import requests
from bs4 import BeautifulSoup
from time import sleep

soup = []
for link in df['links']:
    URL = link

    headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246"}
    # Here the user agent is for Edge browser on windows 10. You can find your browser user agent from the above given link.
    r = requests.get(url=URL, headers=headers)
    # print(r.content)

    soup.append(BeautifulSoup(r.content, 'html5lib')) # If this line causes an error, run 'pip install html5lib' or install html5lib
    # print(soup[-1].prettify())
    sleep(30)

In [89]:
titles = []

for s in soup:
    title = s.find('h1', attrs = {"class": "Headline-headline-2FXIq Headline-black-OogpV ArticleHeader-headline-NlAqj"})
    if title is not None:
        titles.append(title.text) 

In [90]:
len(titles)

10

In [91]:
bodies = []

for s in soup:
    body = ""
    table = s.find("div", attrs = {"class": "TwoColumnsLayout-body-86gsE ArticlePage-body-container-10RhS"})
    if table is not None:
        for row in table.findAll("p",
                                attrs = {"class":"Paragraph-paragraph-2Bgue ArticleBody-para-TD_9x"}):
            body += row.text + "\n"

        bodies.append(body)

In [92]:
len(bodies)

10

In [93]:
df['title'] = pd.Series(titles)
df['body'] = pd.Series(bodies)
df

,sentence,target,sentiment_score,links,source,title,body
0,Doubts grow over GlaxoSmithKline's $6 bln capi...,GlaxoSmithKline,-0.221,https://www.reuters.com/article/gsk-capital-id...,www.reuters.com,Doubts grow over GlaxoSmithKline's $6 bln capi...,* Money may be used to shore up dividend -anal...
1,easyJet leads Britain's FTSE lower as global b...,easyJet,-0.495,https://www.reuters.com/article/markets-stocks...,www.reuters.com,easyJet leads Britain's FTSE lower as global b...,* FTSE 100 down 0.9 pct\n* Appetite for equiti...
2,Imperial Tobacco Adds Quarterly Dividends Afte...,Imperial Tobacco,0.336,https://www.reuters.com/article/imperial-tobac...,www.reuters.com,UPDATE 2-Imperial Tobacco pledges 10 percent d...,"* Raised dividend by 10 pct, pledges same or b..."
3,SABMiller buys Meantime to quench thirst for c...,SABMiller,0.296,https://www.reuters.com/article/us-meantime-m-...,www.reuters.com,SABMiller buys Meantime to quench thirst for c...,LONDON (Reuters) - SABMiller SAB.L is buying B...
4,Retailers Kingfisher and Sports Direct rise in...,Sports Direct,0.459,https://jp.reuters.com/article/markets-stocks-...,jp.reuters.com,Britain's FTSE index gains as retail stocks rally,* Blue-chip FTSE 100 index up 0.2 percent\n* K...
5,"GlaxoSmithKline targets growth across units, s...",GlaxoSmithKline,0.229,https://www.reuters.com/article/gsk-results-id...,www.reuters.com,"GlaxoSmithKline targets growth across units, s...","LONDON, May 6 (Reuters) - GlaxoSmithKline laid..."
6,"RPT-Old Mutual Q1 gross sales beat forecasts, ...",Old Mutual,0.724,https://www.reuters.com/article/old-mutual-res...,www.reuters.com,"RPT-Old Mutual Q1 gross sales beat forecasts, ...","(Repeats to attach to headlines)\nLONDON, May ..."
7,UPDATE 3-BP settles oil spill-related claims w...,BP,-0.121,https://www.reuters.com/article/halliburton-bp...,www.reuters.com,UPDATE 3-BP settles oil spill-related claims w...,(Adds statement from plaintiffs’ attorneys)\nM...
8,UPDATE 1-Cypress Semiconductor offers to buy I...,Integrated Silicon Solution,0.461,https://www.reuters.com/article/integrated-sil...,www.reuters.com,UPDATE 1-Cypress Semiconductor offers to buy I...,"(Adds details, analyst comment, shares)\nMay 1..."
9,"UPDATE 1-Nomura, RBS must pay $806 mln in mort...",RBS,-0.690,https://www.reuters.com/article/fhfa-nomura-hl...,www.reuters.com,"UPDATE 1-Nomura, RBS must pay $806 mln in mort...","(Adds details on order, comment from Nomura)\n..."


In [94]:
import os
# if file does not exist write header 
if not os.path.isfile('fiqa_reuters.csv'):
   df.to_csv('fiqa_reuters.csv', header='column_names', index=False)
else: # else it exists so append without writing the header
   df.to_csv('fiqa_reuters.csv', mode='a', header=False, index=False)

# df.to_csv("file.csv", encoding='utf-8', index=False)